In [52]:
from pygluon.reactors import GluonZReactor
from pygluon.reactors.types import (
    GluonReaction,
    GluonZReactorParameters,
    GluonZReactorState,
    Tokeons,
    GluonUserState
)

### Beta-Decay Plus Test

##### Case 1: q* < q, q* <= q- < q (DEPEG)

In [141]:
# Initial parameters for the reactor.

params = GluonZReactorParameters(
    0.5,
    0.05,
    0.05,
    0.0,
    0.4,
    0.5
)

state = GluonZReactorState(
    10,
    10,
    10,
    0.0,
    0.0
)

reactor = GluonZReactor(params, state)

user_state = GluonUserState(
    0,
    0,
    5
)

n = user_state.neutrons / reactor.state.neutron_circulating_supply
p = user_state.protons / reactor.state.proton_circulating_supply
P = 1.2
qstar = reactor.parameters.critical_neutron_ratio
q = reactor.neutron_ratio(P)

print("normalized_neutron_balance: ", n)
print("normalized_proton_balance: ", p)
print("neutron_target_price: ", P)
print("q*: ", qstar)
print("q: ", q)

normalized_neutron_balance:  0.0
normalized_proton_balance:  0.5
neutron_target_price:  1.2
q*:  0.5
q:  1.2


In [142]:
# Beta-Decay Plus Test
# Case 1: q* < q, q* <= q- < q (DEPEG)

# Intermediate variables.
epsilon = q - qstar
M = 5
m = M / reactor.state.proton_circulating_supply
beta_plus_fee = reactor.parameters.beta_decay_fee_intercept

tau = 1 + epsilon
omega = (1-qstar)*p + q*n

print("epsilon: ", epsilon)
print("reaction_protons: ", M)
print("normalized_reaction_protons: ", m)
print("beta_plus_fee: ", beta_plus_fee)
print("tau: ", tau)
print("omega: ", omega)

# Test the tau lower bound - i.e. epsilon threshold.
tau_lower_bound = (omega*(1-m)) / (p-m) if (m < p) else 1_000_000 
print("tau_lower_bound: ", tau_lower_bound)
print("tau > tau_upper_bound: ", tau > tau_lower_bound)

# Test beta-plus fee bound.
beta_plus_bound = 1 - ((tau/(tau-omega))*((1-p)/(1-m)))
print("beta_plus_bound: ", beta_plus_bound)
print("beta_decay_plus_fee < beta_plus_bound: ", beta_plus_fee < beta_plus_bound)

# Test value preservation.
V = user_state.basecoins + user_state.neutrons*reactor.neutron_price(P) + user_state.protons*reactor.proton_price(P)
print("Initial value: ", V)

beta_decay_plus = reactor.execute(GluonReaction.BETA_DECAY_PLUS, M, P, 1)
qPrime = reactor.neutron_ratio(P)
Nprime = user_state.neutrons + beta_decay_plus.reactor_output
Pprime = user_state.protons - M
Vprime = user_state.basecoins + Nprime*reactor.neutron_price(P) + Pprime*reactor.proton_price(P)

print("PrimeNeutronRatio: ", qPrime)
print("PrimeNeutrons: ", Nprime)
print("PrimeProtons: ", Pprime)
print("Prime value: ", Vprime)
print("Value preserved: ", Vprime <= V)

epsilon:  0.7
reaction_protons:  5
normalized_reaction_protons:  0.5
beta_plus_fee:  0.4
tau:  1.7
omega:  0.25
tau_lower_bound:  1000000
tau > tau_upper_bound:  False
beta_plus_bound:  -0.17241379310344818
beta_decay_plus_fee < beta_plus_bound:  False
Initial value:  1.4705882352941173
PrimeNeutronRatio:  1.35
PrimeNeutrons:  1.2499999999999998
PrimeProtons:  0
Prime value:  0.8108108108108105
Value preserved:  True


##### Case 2: q < q*, q < q+ < q* (PEG)

In [120]:
# Initial parameters for the reactor.

params = GluonZReactorParameters(
    0.99,
    0.05,
    0.05,
    0.0,
    0.25,
    0.5
)

state = GluonZReactorState(
    20,
    10,
    10,
    0.0,
    0.0
)

reactor = GluonZReactor(params, state)

user_state = GluonUserState(
    0,
    0,
    5
)

n = user_state.neutrons / reactor.state.neutron_circulating_supply
p = user_state.protons / reactor.state.proton_circulating_supply
P = 0.8
qstar = reactor.parameters.critical_neutron_ratio
q = reactor.neutron_ratio(P)

print("normalized_neutron_balance: ", n)
print("normalized_proton_balance: ", p)
print("neutron_target_price: ", P)
print("q*: ", qstar)
print("q: ", q)

normalized_neutron_balance:  0.0
normalized_proton_balance:  0.5
neutron_target_price:  0.8
q*:  0.99
q:  0.4


In [121]:
# Beta-Decay Plus Test
# Case 2: q < q*, q < q+ < q* (PEG)

# Intermediate variables.
epsilon = qstar - q
M = 1
m = M / reactor.state.proton_circulating_supply
beta_plus_fee = reactor.parameters.beta_decay_fee_intercept

print("epsilon: ", epsilon)
print("reaction_protons: ", M)
print("normalized_reaction_protons: ", m)
print("beta_plus_fee: ", beta_plus_fee)

# Test value preservation.
V = user_state.basecoins + user_state.neutrons*reactor.neutron_price(P) + user_state.protons*reactor.proton_price(P)
print("Initial value: ", V)

beta_decay_plus = reactor.execute(GluonReaction.BETA_DECAY_PLUS, M, P, 1)
qPrime = reactor.neutron_ratio(P)
Nprime = user_state.neutrons + beta_decay_plus.reactor_output
Pprime = user_state.protons - M
Vprime = user_state.basecoins + Nprime*reactor.neutron_price(P) + Pprime*reactor.proton_price(P)

print("PrimeNeutronRatio: ", qPrime)
print("PrimeNeutrons: ", Nprime)
print("PrimeProtons: ", Pprime)
print("Prime value: ", Vprime)
print("Value preserved: ", Vprime <= V)

epsilon:  0.59
reaction_protons:  1
normalized_reaction_protons:  0.1
beta_plus_fee:  0.25
Initial value:  6.0
PrimeNeutronRatio:  0.445
PrimeNeutrons:  1.1249999999999998
PrimeProtons:  4
Prime value:  5.833333333333332
Value preserved:  True


### Beta-Decay Minus Test

##### Case 1: q* < q, q* <= q- < q (DEPEG)

In [127]:
# Initial parameters for the reactor.

params = GluonZReactorParameters(
    0.5,
    0.05,
    0.05,
    0.0,
    0.1,
    0.5
)

state = GluonZReactorState(
    20,
    10,
    10,
    0.0,
    0.0
)

reactor = GluonZReactor(params, state)

user_state = GluonUserState(
    0,
    5,
    0
)

n = user_state.neutrons / reactor.state.neutron_circulating_supply
p = user_state.protons / reactor.state.proton_circulating_supply
P = 1.2
qstar = reactor.parameters.critical_neutron_ratio
q = reactor.neutron_ratio(P)

print("normalized_neutron_balance: ", n)
print("normalized_proton_balance: ", p)
print("neutron_target_price: ", P)
print("q*: ", qstar)
print("q: ", q)

normalized_neutron_balance:  0.5
normalized_proton_balance:  0.0
neutron_target_price:  1.2
q*:  0.5
q:  0.6


In [128]:
# Beta-Decay Minus Test
# Case 1: q* < q, q* <= q- < q (DEPEG)

# Intermediate variables.
M = 5
m = M / reactor.state.neutron_circulating_supply

epsilon = q - qstar

tau = 1 + epsilon
omega = (1-qstar)*p + q*n

tau_bar = (1-qstar) + q*m
omega_bar = (1-qstar)*p + q*m

beta_minus_fee = reactor.parameters.beta_decay_fee_intercept

print("reaction_neutrons: ", M)
print("normalized_reaction_neutrons: ", m)
print("epsilon: ", epsilon)
print("tau: ", tau)
print("omega: ", omega)
print("tau_bar: ", tau_bar)
print("omega_bar: ", omega_bar)
print("beta_minus_fee: ", beta_minus_fee)

# Test the tau upper bound - i.e. epsilon threshold.
tau_upper_bound = (tau_bar / omega_bar) * omega
print("tau_upper_bound: ", tau_upper_bound)
print("tau < tau_upper_bound: ", tau < tau_upper_bound)

# Test beta-minus fee bound.
beta_minus_bound = 1 - (((1-qstar)*(tau-omega))/((tau*(1-qstar)*(1-p))-((tau-omega)*q*m)))
print("beta_minus_bound: ", beta_minus_bound)
print("beta_decay_minus_fee < beta_minus_bound: ", beta_minus_fee < beta_minus_bound)

# Test value preservation.
V = user_state.basecoins + user_state.neutrons*reactor.neutron_price(P) + user_state.protons*reactor.proton_price(P)
print("Initial value: ", V)

beta_decay_minus = reactor.execute(GluonReaction.BETA_DECAY_MINUS, M, P, 1)
Pprime = user_state.protons + beta_decay_minus.reactor_output
Nprime = user_state.neutrons - M
Vprime = user_state.basecoins + Nprime*reactor.neutron_price(P) + Pprime*reactor.proton_price(P)

print("PrimeNeutrons: ", Nprime)
print("PrimeProtons: ", Pprime)
print("Prime value: ", Vprime)
print("Value preserved: ", Vprime <= V)


reaction_neutrons:  5
normalized_reaction_neutrons:  0.5
epsilon:  0.09999999999999998
tau:  1.1
omega:  0.3
tau_bar:  0.8
omega_bar:  0.3
beta_minus_fee:  0.1
tau_upper_bound:  0.8
tau < tau_upper_bound:  False
beta_minus_bound:  -0.29032258064516103
beta_decay_minus_fee < beta_minus_bound:  False
Initial value:  5.454545454545454
PrimeNeutrons:  0
PrimeProtons:  5.399999999999999
Prime value:  4.909090909090908
Value preserved:  True
